In [ ]:
# connect to postgresql db usingenvironment variable read from vars.env 

import warnings
warnings.filterwarnings("ignore")

import os
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# read environment variables from vars.env
from dotenv import load_dotenv
load_dotenv("../vars.env")

# connect to postgresql db on localhost, post 5432, using user and password from vars.env

import psycopg2
import os

# Define the database credentials
db_host = os.getenv("POSTGRES_HOST")
db_name = os.getenv("POSTGRES_DB")
db_user = os.getenv("POSTGRES_USER")
db_password = os.getenv("POSTGRES_PWD")



In [ ]:
# I want to join some of the datasets into one in database  also each sample is linked with dataset when correcting it the samples should be linked with the new dataset

# "French(Dorsaf)", "Italian(Martina)", "Spanish(Violeta)", "English(Melynda)"

# all datasets include French(Dorsaf) should be named as French(Dorsaf)
# get the base dataset which has the name French(Dorsaf) and update all the datasets which include French(Dorsaf) to French(Dorsaf)
#

for dataset_name in ["French(Dorsaf)", "Italian(Martina)", "Spanish(Violeta)", "English(Melynda)"]:
    try:
        # Establish a connection to the database
        conn = psycopg2.connect(
            host=db_host,
            dbname=db_name,
            user=db_user,
            password=db_password
        )

        # Create a cursor to perform database operations
        cursor = conn.cursor()

        # Assuming you have a table named "datasets" with columns "id" and "name"
        # Fetch the dataset with the name "French(Dorsaf)"
        cursor.execute(f"SELECT * FROM dataset WHERE name = '{dataset_name}';")
        base_dataset = cursor.fetchone()
        
        
        
        if base_dataset:
            print(f"Base dataset found: {base_dataset[1]}")
            base_id = base_dataset[0]
            base_name = base_dataset[1]
            
            # get all datasets that include "French(Dorsaf)"
            cursor.execute(f"SELECT * FROM dataset WHERE name LIKE '%{dataset_name}%';")
            datasets = cursor.fetchall()

            for dataset in datasets:
                if dataset[1] == base_name:
                    continue
                print(f"Updating dataset {dataset[1]}...")
                # update all samples which has dataset_id = dataset.id to base_id
                cursor.execute(f"UPDATE sample SET dataset_id = {base_id} WHERE dataset_id = {dataset[0]};")
                conn.commit()
                print(f"Deleting dataset {dataset[1]}...")

                # updaee annotator_dataset table with new dataset_id if not annotator id and dataset id pair exists
                try:
                    cursor.execute(f"UPDATE annotator_dataset SET dataset_id = {base_id} WHERE dataset_id = {dataset[0]};")
                    conn.commit()
                except psycopg2.Error as e:
                    print("Error updating annotator_dataset table: ", e)
                # delete the dataset
                cursor.execute(f"DELETE FROM dataset WHERE id = {dataset[0]};")
                conn.commit()


            print("Dataset names updated successfully.")
        else:
            print("Base dataset 'French(Dorsaf)' not found.")

        # # Close the cursor and connection
        cursor.close()
        conn.close()

    except psycopg2.Error as e:
        print(f"Error connecting to the database: {e}")




In [ ]:
base_id